In [1]:
import pandas as pd
import pickle
import numpy as np
from Constants import Const
import torch
import torch.nn as nn
import cv2
from pointcloud_utils import *
import open3d as o3d

In [15]:
{i['id']: i for i in plist}

{1: {'id': 1,
  'Spacing': (0.9765625, 0.9765625, 3.0),
  'Volumes': array([26.63135529,  0.        , 33.3070755 ,  8.60452652,  2.96831131,
          4.45365906, 22.89772034,  5.17606735,  7.52210617,  7.9369545 ,
          2.08854675,  1.50918961, 28.38373184, 26.48115158,  2.2816658 ,
          0.        , 16.41273499,  9.59634781,  8.70943069, 89.06841278,
         90.30342102]),
  'ArrayDicom': array([[[-1024., -1024., -1024., ..., -1024., -1024., -1024.],
          [-1024., -1024., -1024., ..., -1024., -1024., -1024.],
          [-1024., -1024., -1024., ..., -1024., -1024., -1024.],
          ...,
          [-1024., -1024., -1024., ..., -1024., -1024., -1024.],
          [-1024., -1024., -1024., ..., -1024., -1024., -1024.],
          [-1024., -1024., -1024., ..., -1024., -1024., -1024.]],
  
         [[-1024., -1024., -1024., ..., -1024., -1024., -1024.],
          [-1024., -1024., -1024., ..., -1024., -1024., -1024.],
          [-1024., -1024., -1024., ..., -1024., -1024., -102

In [2]:
with open(Const.data_dir+'processed_dicoms.p','rb') as f:
    plist = pickle.load( f ) 
len(plist)

5

In [3]:
# def vectorize_contours(pdict,rois=None,min_size = 2):
#     contours = pdict['contours']
#     contours = pcloud_to_concave_hull(contours)
#     if rois is None:
#         rois = sorted(contours.keys())
#     array = []
#     for roi in rois:
#         contour = contours.get(roi)
#         if contour is None or contour.shape[0] < min_size:
#             contour = np.zeros((min_size,3))
#         array.append(contour)
#     #returns a list of variable size pointclouds
#     return array
        
# def downsample_pcloud(xyz,size=2):
# #     print(xyz.shape)
#     pcd = o3d.geometry.PointCloud()
#     pcd.points = o3d.utility.Vector3dVector(xyz)
#     pcd2 = pcd.voxel_down_sample(size)
#     xyz_new = np.asarray(pcd2.points)
# #     print(xyz_new.shape[0],xyz_new.shape[0]/xyz.shape[0],1/size)
#     return xyz_new

# def resize_pcloud(xyz, npoints = 2000,jitter=.1):
#     #voxel downsample
#     if xyz.shape[0] < 2:
#         return np.zeros((npoints,3))
#     correction = 0
#     prev_size = xyz.shape[0]
#     while xyz.shape[0] > npoints:
#         xyz = downsample_pcloud(xyz,(xyz.shape[0]/npoints)+correction)
#         if xyz.shape[0] >= prev_size:
#             correction += .1
#         prev_size = xyz.shape[0]
# #         print('---')
# #     print(xyz.shape)
#     #padd pointcloud with the centroid of the point + some jitter
#     diff = npoints - xyz.shape[0]
#     pad = np.zeros((diff,3))
#     pad[:] = cloud_centroid(xyz)
#     if jitter:
#         pad = pad + (np.random.random(pad.shape) - .5)*jitter
#     new_points = np.vstack([xyz,pad])
# #     print('new points',new_points.shape)
# #     print('------------------------')
#     return new_points


# def vectorize_plist_pclouds(plist,rois=None,size=1000):
#     #how to make htis a uniform shape
#     arrays = []
#     if rois is None:
#         rois = list(plist[0]['roi_mask_map'].values())
#     for patient in plist:
#         arr = vectorize_contours(patient,rois=rois)
#         print(arr[0].shape)
#         if size is not None and size > 0:
#             arr = [resize_pcloud(a,size) for a in arr]
#             arr = np.stack(arr)
#         arrays.append(arr)
#     if size is not None and size > 0:
#         arrays = np.stack(arrays)
#     return arrays
    
# test = vectorize_plist_pclouds(plist[2:])
# test.shape

In [4]:
# import matplotlib.pyplot as plt
# import matplotlib as mpl
# import plotly.graph_objects as go

# def plot_pcloud_array(array,figsize=(8,8)):
#     fig = plt.figure(figsize=figsize,facecolor='w')
#     ax = fig.add_subplot(projection='3d')
#     ax.set_facecolor('w')
#     ax.grid(False)
#     colors = mpl.colormaps['tab20']
#     assert(array.ndim == 3)
#     assert(array.shape[-1] == 3)
#     for pointcloud in array:
#         size = .2
#         ax.scatter(pointcloud[:,0],pointcloud[:,1],pointcloud[:,2],s=size,color='grey')
#     return ax

# def plotly_pcloud_array(array):
#     data = []
#     assert(array.ndim == 3)
#     assert(array.shape[-1] == 3)
#     for points in array:
#         if points.shape[0] < 5:
#             continue
#         color='gray'
#         subplot = go.Scatter3d(
#             x=points[:,0], y=points[:,1], z=points[:,2], 
#             mode='markers',
#             marker=dict(size=1.5,color=color)
#         )
#         data.append(subplot)
#     fig = go.Figure(
#         data = data,
#         layout=dict(
#             scene=dict(
#                 xaxis=dict(visible=False),
#                 yaxis=dict(visible=False),
#                 zaxis=dict(visible=False)
#             )
#         )
#     )
#     return fig

# plotly_pcloud_array(test[0])

In [5]:
#code I tried to get convolutional network to work on

In [6]:
def get_array(patient):
    #extracts an array of image slices with contours
    values = patient['ArrayDicom']
    mask = patient['mask']
    
    #make stuff that isn't a contour 0
    good_values = values * (mask > 0)
    
    #trim slices with no contours
    not_empty = good_values.sum(axis=1).sum(axis=1) > 0.01
    first_true = -1
    last_true = -1
    for pos,boolean in enumerate(not_empty):
        if boolean:
            if first_true < 0:
                first_true = pos
            else:
                last_true = max(last_true,pos)
    return good_values[first_true:last_true+1], mask[first_true:last_true]

def pad_height(array3d,height,fill=0):
    shape = array3d.shape[1:]
    npads=height- array3d.shape[0] 
    assert(npads >= 0)
    if npads == 0:
        return array3d
    
    pad = np.full((npads,shape[0],shape[1]),fill)
    return np.vstack([array3d,pad])

def arrays_from_plist(plist, pad=0,scale= True, add_channel=False,square=False,img_size=56):
   #returns  arrays of image vlaues n_patients x n_images x width x height
    # x is input images values, y is the class labels
    #img_size will resize images. uses linear for x and nearest neighbors when estimating roi membership so edges might be a bit wonky
    x,y = zip(*[get_array(p) for p in plist])
    x = list(x)
    y = list(y)
    if square:
        max_height = img_size
    else:
        max_height = np.max([xx.shape[0] for xx in x ])
    x = [pad_height(xx,max_height,pad) for xx in x]
    y =  [pad_height(yy,max_height,pad) for yy in y]
    #make it square
    #when resizeing y (organ index labels), i thing inter_nearest uses a nearest neighbors approach so it returns ints also
    if img_size is not None:
        x = [np.stack([cv2.resize(xxx, (img_size,img_size)) for xxx in xx]) for xx in x]
        y = [np.stack([cv2.resize(yyy, (img_size,img_size),interpolation=cv2.INTER_NEAREST) for yyy in yy]) for yy in y]
    x = np.stack(x)
    y = np.stack(y)
    #scale to be 0-1
    if scale:
        x = (x-x.min())/(x.max() - x.min())
    #adds a dimension so  htat the model fits for a 3d convolution for pytorch
    if add_channel:
        s = x.shape
        x = x.reshape((s[0],1,s[1],s[2],s[3]))
    return x,y

test = arrays_from_plist(plist,add_channel=True)
print([t.shape for t in test])
print(np.unique(test[1]))
del test

[(5, 1, 104, 56, 56), (5, 104, 56, 56)]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 17 18 19 20 21]


In [7]:
len(plist)

5

In [8]:
import torch
import torch.nn as nn

class C3Encoder(nn.Module):
    def __init__(self,
                 hidden_dims = [16,32,64],
                 fc_dims=[128], #last fc_dim will be 
                 conv_kernel_size=3,
                 channels=1,
                ):

        super(C3Encoder, self).__init__()
        groups = []
        curr_in = channels
        for hidden_dim in hidden_dims:
            conv = self.make_conv_group(curr_in,hidden_dim,conv_kernel_size)
            curr_in = hidden_dim
            groups.append(conv)
        self.groups= nn.ModuleList(groups)
        
        self.init_fc_layers(fc_dims)
        
        
    def make_conv_group(self,curr_in, curr_out,kernel_size):
        conv = nn.Sequential(
            nn.Conv3d(curr_in,curr_out,kernel_size=kernel_size,padding=1),
            nn.BatchNorm3d(curr_out),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(1, 2, 2)),
        )
        return conv
        

    def init_fc_layers(self,fc_dims):
        fc_layers = []
        for fc_dim in fc_dims:
            group = self.make_linear_group(fc_dim)
            fc_layers.append(group)
            
        self.fc_layers = nn.ModuleList(fc_layers)
        self.output_size = fc_dims[-1]
        
    def make_linear_group(self, curr_out,**kwargs):
        return nn.LazyLinear(curr_out,**kwargs)
    
    def forward(self, x):
#         x = torch.from_numpy(x).float().cuda()
        for group in self.groups:
            x = group(x)
        out = x.view(x.size(0), -1)
        for layer in self.fc_layers:
            out = layer(out)
        return out
    

    
class C3Decoder(nn.Module):
    def __init__(self,
                fc_dims=[128,128],
                 image_output_size = [104,128,128],
                 num_classes = 10,
                ):
        super(C3Decoder,self).__init__()
        
        self.init_fc_layers(fc_dims)
        #add in a hidden layer with the right size so we end up with enough outputs 
        #this will just be an image unraveled
        output_shape = np.prod(image_output_size)
        self.output_shape = image_output_size
        #so I can use it as the equivalent of the channels section (this is a bad idea)
        
        self.num_classes = num_classes 
        self.final_linear = nn.Linear(fc_dims[-1],output_shape)
        self.orient = lambda x: x.view(x.shape[0],1,-1)
        self.conv1d = nn.Conv1d(1,num_classes,kernel_size=1)
        self.softmax = nn.Softmax(dim=1)
        self.out_shape = image_output_size
    
    def init_fc_layers(self,fc_dims):
        fc_layers = []
        for fc_dim in fc_dims:
            group = self.make_linear_group(fc_dim)
            fc_layers.append(group)
            
        self.fc_layers = nn.ModuleList(fc_layers)
        self.output_size = fc_dims[-1]
        
    def make_linear_group(self, curr_out,**kwargs):
        return nn.LazyLinear(curr_out,**kwargs)
    
    def forward(self,x):
#         print('decode start',x.shape)
        for layer in self.fc_layers:
            x = layer(x)
#         print('after linear',x.shape)
        x = self.final_linear(x)
#         print('fl',x.shape)
        x = self.orient(x)
#         print('orient', x.shape)
        x = self.conv1d(x)
#         print('conv1d',x.shape)
        x = self.softmax(x)
#         print('softmax',x.shape)
        ##this was if I want to ouptut a single class
#         x = torch.argmax(x,dim=1)
#         return x.view(x.shape[0],self.out_shape[0],self.out_shape[1],self.out_shape[2])
        #returnx ndim x nclasses x nimages x width x height so I can use crossentropy loss
        return x.view(x.shape[0],-1,self.out_shape[0],self.out_shape[1],self.out_shape[2])

class C3AutoEncoder(nn.Module):
    #currently this is a cnn encoder + linear classifier that predicts if a pixel belongs to a part of an roi
    #you need to specify the number of rois
    def __init__(self,
                 example_data, #example imput data, takes last 3 rows to get output image size
                 encoder=None,
                 decoder=None,
                 num_classes=10,
                ):
        super(C3AutoEncoder,self).__init__()
        if encoder is None:
            encoder = C3Encoder()
            
        while example_data.ndim > 3:
            example_data = example_data[0]
        if decoder is None:
            decoder = C3Decoder(image_output_size=example_data.shape,num_classes=num_classes)
            
        self.encoder=encoder
        self.decoder=decoder
        self.output_shape = example_data.shape
        self.num_classes = num_classes
        
    def forward(self,x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
testx,testy = arrays_from_plist(plist,add_channel=True,img_size=56)
test = C3AutoEncoder(testx[0]).cuda()
test
print(test( torch.from_numpy(testx[1:3]).float().cuda() ).shape)
del test, testx, testy
torch.cuda.empty_cache()

/home/evl/andrew/anaconda3/envs/Qubbed/lib/python3.7/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 10, 104, 56, 56])


In [9]:
import torchio as tio

class NumpyImageDataset(torch.utils.data.Dataset):
    
    def __init__(self,x,y, transforms = True):
        self.data = torch.from_numpy(x).float()
        self.targets = torch.from_numpy(y).long()
        if transforms:
            transforms = tio.Compose([tio.RandomNoise(p=.5),tio.RandomAffine(p=.5)])
        self.transforms = transforms
        
    def __getitem__(self,index):
        x = self.data[index]
        y = self.targets[index]
        
        if self.transforms:
            x = self.transforms(x)
        return x, y
    
    def __len__(self):
        return len(self.data)

def train_torch(plist,model=None,img_size=56,batch_size=1,epochs=1000,patience=10):
    torch.cuda.empty_cache()
    x,y = arrays_from_plist(plist,add_channel=True,img_size=img_size)
    nclasses = len(plist[0]['roi_mask_map'].keys()) + 1
    if model is None:
        model = C3AutoEncoder(x[0,0],num_classes=nclasses)
    model = model.cuda()
    
    ce_loss = nn.CrossEntropyLoss().cuda()
    optimizer = torch.optim.Adam(model.parameters(),lr=.001)
    
    def get_loss(ypred,ytrain):
        ytrain = ytrain.view(-1)
        ypred = torch.moveaxis(ypred,1,-1).view(ytrain.shape[0],nclasses)
        loss = ce_loss(ypred.float(),ytrain.long())
        return loss
    
    def get_tp(ypred,y):
        y = y.view(-1)
        ypred = torch.moveaxis(ypred,1,-1).view(y.shape[0],nclasses)
        ypred = torch.argmin(ypred,dim=1)
        correct = torch.sum(ypred == y)
        return correct/len(y)
    
    def get_loader(xx,yy,train):
        dataset = NumpyImageDataset(xx, yy,transforms=train)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size = batch_size,shuffle=train)
        return dataloader
    
    train_dataloader = get_loader(x[2:],y[2:],True)
    val_dataloader = get_loader(x[:2],y[:2],True)
    
    
    def train_step(xtrain,ytrain):
        model.train()
        optimizer.zero_grad()
        xtrain = xtrain.cuda()
        ytrain = ytrain.cuda()
        ypred = model(xtrain)
        loss = get_loss(ypred,ytrain)
        loss.backward()
        optimizer.step()
        return loss
    
    def val_step(xval,yval):
        model.eval()
        with torch.no_grad():
            xval = xval.cuda()
            yval = yval.cuda()
            ypred = model(xval)
            loss = get_loss(ypred,yval)
            true_positives = get_tp(ypred,yval)
            return loss, true_positives
        
    best_val_loss = 100000000
    steps_since_improvement = 0
    for epoch in range(epochs):
        epoch_loss = 0
        for xtrain, ytrain in train_dataloader:
            loss = train_step(xtrain,ytrain)
            epoch_loss  += loss.item()
            
        epoch_val_loss = 0
        val_accuracy = 0
        for xval, yval in val_dataloader:
            loss,val_tp = val_step(xval,yval)
            epoch_val_loss += loss.item()
            val_accuracy += val_tp.item()
        val_accuracy = val_accuracy/len(val_dataloader)
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            steps_since_improvement = 0
        else:
            steps_since_improvement += 1
        print('epoch',epoch,'train loss',epoch_loss, 'val loss',epoch_val_loss,'val acc',val_accuracy,'remaining_tries',patience - steps_since_improvement)
        if steps_since_improvement > patience:
            break
    return model

train_torch(plist)

/home/evl/andrew/anaconda3/envs/Qubbed/lib/python3.7/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


epoch 0 train loss 9.180432796478271 val loss 5.8978331089019775 val acc 0.00021922832820564508 remaining_tries 10
epoch 1 train loss 9.197551012039185 val loss 5.918818712234497 val acc 0.00015330652240663767 remaining_tries 9
epoch 2 train loss 9.009192705154419 val loss 5.836611032485962 val acc 0.00020849686552537605 remaining_tries 10
epoch 3 train loss 8.780360460281372 val loss 5.80911922454834 val acc 0.0001901000869111158 remaining_tries 10
epoch 4 train loss 8.727834224700928 val loss 5.77264142036438 val acc 0.00017783555813366547 remaining_tries 10
epoch 5 train loss 8.630842208862305 val loss 5.700827598571777 val acc 0.00019316621182952076 remaining_tries 10
epoch 6 train loss 8.600394487380981 val loss 5.6377198696136475 val acc 0.00019163314573233947 remaining_tries 10
epoch 7 train loss 8.472360849380493 val loss 5.601292133331299 val acc 0.00020696379942819476 remaining_tries 10
epoch 8 train loss 8.374368667602539 val loss 5.499507427215576 val acc 0.0015529950032941

epoch 73 train loss 6.678686618804932 val loss 4.452768087387085 val acc 0.055929284542798996 remaining_tries 10
epoch 74 train loss 6.678674697875977 val loss 4.452758550643921 val acc 0.056430596858263016 remaining_tries 10
epoch 75 train loss 6.678659677505493 val loss 4.4527668952941895 val acc 0.05585569702088833 remaining_tries 9
epoch 76 train loss 6.678627967834473 val loss 4.452791452407837 val acc 0.05312224105000496 remaining_tries 8
epoch 77 train loss 6.67858624458313 val loss 4.452703475952148 val acc 0.05636467598378658 remaining_tries 10
epoch 78 train loss 6.678558588027954 val loss 4.452669143676758 val acc 0.05635547637939453 remaining_tries 10
epoch 79 train loss 6.678510427474976 val loss 4.4526567459106445 val acc 0.056338611990213394 remaining_tries 10
epoch 80 train loss 6.678497076034546 val loss 4.45264196395874 val acc 0.056301819160580635 remaining_tries 10
epoch 81 train loss 6.678468942642212 val loss 4.452619314193726 val acc 0.05633401311933994 remaining

epoch 146 train loss 6.677164554595947 val loss 4.451765537261963 val acc 0.05613011494278908 remaining_tries 10
epoch 147 train loss 6.677166700363159 val loss 4.4518609046936035 val acc 0.05613011494278908 remaining_tries 9
epoch 148 train loss 6.677158355712891 val loss 4.451874017715454 val acc 0.05613011494278908 remaining_tries 8
epoch 149 train loss 6.677156925201416 val loss 4.451761960983276 val acc 0.05613011494278908 remaining_tries 10
epoch 150 train loss 6.6771533489227295 val loss 4.451761484146118 val acc 0.05613011494278908 remaining_tries 10
epoch 151 train loss 6.67715311050415 val loss 4.451767444610596 val acc 0.05613011494278908 remaining_tries 9
epoch 152 train loss 6.677152633666992 val loss 4.451763391494751 val acc 0.05613011494278908 remaining_tries 8
epoch 153 train loss 6.677151918411255 val loss 4.451760530471802 val acc 0.05613011494278908 remaining_tries 10
epoch 154 train loss 6.677156209945679 val loss 4.45176362991333 val acc 0.05613011494278908 remain

C3AutoEncoder(
  (encoder): C3Encoder(
    (groups): ModuleList(
      (0): Sequential(
        (0): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): MaxPool3d(kernel_size=(2, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
      )
      (1): Sequential(
        (0): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): MaxPool3d(kernel_size=(2, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
      )
      (2): Sequential(
        (0): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): MaxPool3d(kernel_size=(2, 2

In [10]:
torch.cuda.is_available()

True

In [11]:
torch.device('cuda')

device(type='cuda')